# Лабораторная работа №1
## Выполнил: студент группы БВТ2001 Хайдаров Адель Фанисович
### Цель работы: получение практических навыков по программированию в Win32 API.

### Вспомогательные функции

In [ ]:
open System.Runtime.InteropServices
open System.Text

type w = uint16
type dw = uint
type ul = uint
type ull = uint64
type lps = StringBuilder

let mutable maxLength = 261u
let mutable size = maxLength
let mutable sb = lps ()

let dispLast cap =
    display($"{cap}{sb}") |> ignore
    sb <- int <| size |> lps; size <- maxLength

let dColor color (value: int) =
    let t = value.ToString "x6"
    HTML($"""{color}: <b style="color:#{t}">{color}</b> (#{t})""")

let rStr: seq<byte> -> string = Seq.map char >> String.Concat
let rInt (t: byte []): int = int t[0]
let rBool t = (rInt t) > 0

let sToDHMS ct = ct / 86400, ct / 3600 % 24, ct / 60 % 60, ct % 60

#### Подключение библиотеки для получения актуальной информации о версии ОС, так как Win32 API не поддерживает Windows 11:

In [ ]:
#r "System.Management.dll"
open System.Management

let m_dll = new ManagementObject("Win32_OperatingSystem=@")

### Функции Win32 API
Ниже представлены использованные функции Win32 API для получения той или иной информации о системе и для задания некоторых параметров

In [ ]:
[<DllImport("kernel32.dll", CharSet = CharSet.Unicode)>] extern void GetComputerName(lps buffer, dw& size)
[<DllImport("kernel32.dll", CharSet = CharSet.Unicode)>] extern void GetSystemDirectory(lps buffer, dw& size)
[<DllImport("kernel32.dll", CharSet = CharSet.Unicode)>] extern void GetWindowsDirectory(lps buffer, dw& size)
[<DllImport("kernel32.dll", CharSet = CharSet.Unicode)>] extern void GetTempPath(dw size, lps buffer)

type DateTime = struct
    val year: w; val month: w; val dayOfWeek: w; val day: w
    val hour: w; val minute: w; val second: w
end
[<DllImport("kernel32.dll")>] extern void GetSystemTime(DateTime& st)
[<DllImport("kernel32.dll")>] extern void GetLocalTime(DateTime& st)

type SomeMetrics = | ScreenW = 0 | ScreenH = 1 | ScrollbarW = 3 | FScreenW = 16 | FScreenH = 17 | TButtonW = 30 | IsSlow = 73

type Colors = | Window = 5 | WindowText = 8 | Highlight = 24 | Hotlight = 26 | Frame = 6

type P = | ScreenReader = 0x46 | Wallpaper = 0x73 | KillTimeout = 0x7A | NotRespondingTimeout = 0x7C

[<DllImport("user32.dll")>] extern int GetSystemMetrics(SomeMetrics nIndex)
[<DllImport("user32.dll")>] extern bool SystemParametersInfo(P action, uint uiParam, byte [] pv, uint fWinIni)
[<DllImport("user32.dll")>] extern int GetSysColor(Colors nIndex)
[<DllImport("user32.dll")>] extern bool SetSysColors(int cEl, Colors [] elements, int [] color)

let getParam id getter =
    let mutable g: byte [] = int <| size |> Array.zeroCreate
    if SystemParametersInfo(id, size, g, 0u) |> not then failwith ""
    getter g

[<DllImport("secur32.dll", CharSet = CharSet.Unicode)>] extern void GetUserNameEx(dw f, lps buffer, dw& size)

type OsVersionInfo = struct
    val mutable size: ul
    val major: ul; val minor: ul; val build: ul; val platform: ul
end
[<DllImport("ntdll.dll")>] extern void RtlGetVersion(OsVersionInfo& info)

### Выполнение

#### Имя компьютера и полное имя пользователя:

In [ ]:
GetComputerName(sb, &size); dispLast "Имя компьютера: "
GetUserNameEx(3u, sb, &size); dispLast "Имя пользователя: "

Имя компьютера: A-PC

Имя пользователя: Adil Haidar

#### Пути к системным каталогам Windows:

In [ ]:
GetWindowsDirectory(sb, &size); dispLast "Каталог Windows: "
GetSystemDirectory(sb, &size); dispLast "Каталог System32: "
GetTempPath(size, sb); dispLast "Каталог c временными файлами: "

Каталог Windows: C:\Windows

Каталог System32: C:\Windows\system32

Каталог c временными файлами: C:\Users\adel_\AppData\Local\Temp\

#### Версия установленной ОС:

In [ ]:
let mutable v = OsVersionInfo()
display "Данные, полученные с помощью RtlGetVersion:"
RtlGetVersion &v; display v
$"""Версия установленной операционной системы:{"\n"}{m_dll["Caption"]}"""

Данные, полученные с помощью RtlGetVersion:

major,minor,build,platform,size
10,0,22000,2,0


Версия установленной операционной системы:
Microsoft Windows 11 Pro

#### Системные метрики:

In [ ]:
display $"Размер интерфейса ОС: \
          {GetSystemMetrics SomeMetrics.ScreenW}x{GetSystemMetrics SomeMetrics.ScreenH}"
display $"Размер развёрнутого приложения: \
          {GetSystemMetrics SomeMetrics.FScreenW}x{GetSystemMetrics SomeMetrics.FScreenH}"
display $"Ширина заголовочной кнопкиф: {GetSystemMetrics SomeMetrics.TButtonW}px"
display $"Ширина полосы прокрутки: {GetSystemMetrics SomeMetrics.ScrollbarW}px"
display $"Наличие несовместимого процессора: {GetSystemMetrics SomeMetrics.IsSlow > 0}"

Размер интерфейса ОС: 1536x864

Размер развёрнутого приложения: 1536x841

Ширина заголовочной кнопкиф: 36px

Ширина полосы прокрутки: 17px

Наличие несовместимого процессора: False

#### Системные параметры:

In [ ]:
display $"Таймаут остановки зависнувшей программы: {getParam P.NotRespondingTimeout rInt} мс"
display $"Таймаут остановки зависнувшей службы: {getParam P.KillTimeout rInt} мс"
display $"""Экранный диктор в{if getParam P.ScreenReader rBool then "" else "ы"}ключен"""
display $"Путь к обоям рабочего стола: {getParam P.Wallpaper rStr}"

Таймаут остановки зависнувшей программы: 136 мс

Таймаут остановки зависнувшей службы: 136 мс

Экранный диктор выключен

Путь к обоям рабочего стола: C:\Windows\Web\Wallpaper\ThemeA\img21.jpg                                                                                                                                                                                                                            

#### Получение некоторых системных цветов, установка кастомных и откат изменений:

In [ ]:
let s = seq { 
    display "Изначальные цвета:" |> ignore
    for v in Enum.GetValues<Colors>() |> Seq.cast<Colors> do 
        let t = v, GetSysColor v
        dColor <|| t |> display |> ignore
        t
}

let oldColors, newColors = s |> Seq.toArray, [|Colors.Frame, 0x7744ff|]
let n1, n2 = oldColors |> Array.unzip, newColors |> Array.unzip

display "\nНовые цвета: "
newColors |> Array.iter (fun t -> dColor <|| t |> display |> ignore)

let result = SetSysColors(newColors.Length, fst n2, snd n2)
display $"""Новые цвета {if result then "" else "не "}установлены"""

SetSysColors(oldColors.Length, fst n1, snd n1) |> ignore // возврат системных цветов

Изначальные цвета:

Window: Window (#ffffff)

Frame: Frame (#646464)

WindowText: WindowText (#000000)

Highlight: Highlight (#e1ffff)

Hotlight: Hotlight (#cc6600)


Новые цвета: 

Frame: Frame (#7744ff)

Новые цвета установлены

#### Получение системного и местного времени:

In [ ]:
let mutable v = DateTime()
GetSystemTime &v; display $"Дата: %02d{v.day}.%02d{v.month}.{v.year} \
                            Время: %02d{v.hour}:%02d{v.minute}:%02d{v.second}"
GetLocalTime &v; $"Местное время: %02d{v.hour}:%02d{v.minute}:%02d{v.second}"

Дата: 19.09.2022 Время: 18:55:34

Местное время: 21:55:34

### 4 дополнительные API-функции

In [ ]:
[<DllImport("kernel32.dll")>] extern ull GetTickCount64()
type PowerStatus = struct
    val ac: byte; val battery: byte; val bPercent: uint8
    val saverOn: byte; val lt: dw; val flt: dw
end
[<DllImport("kernel32.dll")>] extern bool GetSystemPowerStatus(PowerStatus& status)

type MemoryStatus = struct
    val mutable length: dw
    val load: dw; val total: ull; val available: ull
    val pfTotal: ull; val pfAvail: ull
    val vTotal: ull; val vAvail: ull; val vAvailExt: ull
end
[<DllImport("kernel32.dll")>] extern bool GlobalMemoryStatusEx(MemoryStatus& status)

let currentBattery () =
    let mutable status = PowerStatus()
    if GetSystemPowerStatus &status |> not then failwith ""
    status

let currentMemory () =
    let mutable status = MemoryStatus(length = uint sizeof<MemoryStatus>)
    if GlobalMemoryStatusEx &status |> not then failwith ""
    status

[<DllImport("kernelbase.dll")>] extern bool GetIntegratedDisplaySize(double& sizeInInches)

#### Информация об электропитании и о батарее:

In [ ]:
let power = currentBattery ()
display $"""Питание: от {if (int power.ac) > 0 then "сети" else "батареи"}"""
display $"""Заряд: {power.bPercent}{'%'}"""
display $"""Экономия энергии: в{if (int power.saverOn) > 0 then "" else "ы"}ключена"""
let _, h, m, s = power.lt |> int |> sToDHMS
"Оставшееся время от аккумулятора: " + if s <> -1 then $"{h} ч {m} мин" else "неизвестно"

Питание: от сети

Заряд: 99%

Экономия энергии: выключена

Оставшееся время от аккумулятора: неизвестно

#### Информация об использовании оперативной памяти:

In [ ]:
let usage = currentMemory ()
$"Загруженность оперативной памяти: {usage.load}{'%'}"

Загруженность оперативной памяти: 71%

#### Получение размера (диагонали) встроенного дисплея ноутбука:

In [ ]:
let mutable din = 0.
GetIntegratedDisplaySize &din
$"Диагональ интегрированного дисплея: %.02f{din} дюйма"

Диагональ интегрированного дисплея: 15.53 дюйма

#### Получение времени, прошедшего с момента загрузки системы:

In [ ]:
let time = GetTickCount64 () / 1000UL |> int
let d, h, m, s = sToDHMS time
$"Время с последнего запуска {d} дн {h} час {m} мин {s} сек"

Время с последнего запуска 1 дн 15 час 39 мин 22 сек

# Вывод

В результате лабораторной работы мной было получено несколько видов информации о системе, используя средства Win32 API.